In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pmdarima as pm
import random
import importlib
import mpt_functions as mpt
importlib.reload(mpt)

<module 'mpt_functions' from 'd:\\Code\\GradProject-Bilge\\mpt_functions.py'>

In [2]:
path = "C:/Users/bilge/Downloads/MetroPT3(AirCompressor).csv"
df = mpt.get_the_data_and_convert_datetime(path)
df = mpt.drop_a_column(df, "Unnamed: 0")
df.head()

,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses
0,2020-02-01 00:00:00,-0.012,9.358,9.340,-0.024,9.358,53.600,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
1,2020-02-01 00:00:10,-0.014,9.348,9.332,-0.022,9.348,53.675,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
2,2020-02-01 00:00:19,-0.012,9.338,9.322,-0.022,9.338,53.600,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
3,2020-02-01 00:00:29,-0.012,9.328,9.312,-0.022,9.328,53.425,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
4,2020-02-01 00:00:39,-0.012,9.318,9.302,-0.022,9.318,53.475,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0


In [ ]:
# this should be checked

f1_start = "2020-04-18 00:00:00"
f1_finish = "2020-04-18 23:59:00"
f2_start = "2020-05-29 23:30:00"
f2_finish = "2020-05-30 06:00:00"
f3_start = "2020-06-05 10:00:00"
f3_finish = "2020-06-07 14:30:00"
f4_start = "2020-07-15 14:30:00"
f4_finish = "2020-07-15 19:00:00"

In [ ]:
# # df2 is the new version of df1 with "condition" column added.

df2 = df.assign(condition=0)
df2.head()

In [ ]:
df2a = mpt.change_the_values_by_applying_a_time_filter(df2, f1_start, f1_finish, "condition", 1)
df2b = mpt.change_the_values_by_applying_a_time_filter(df2a, f2_start, f2_finish, "condition", 1)
df2c = mpt.change_the_values_by_applying_a_time_filter(df2b, f3_start, f3_finish, "condition", 1)
df2 = mpt.change_the_values_by_applying_a_time_filter(df2c, f4_start, f4_finish, "condition", 1)

print(df2["condition"].value_counts())


plt.bar([0, 1], df2["condition"].value_counts())
plt.title('Conditions')
plt.xlabel('Condition')
plt.ylabel('Value')
plt.show()

In [ ]:
print(df2.isna().sum())  # Count NaN values
print((df2 == float('inf')).sum())  # Count positive infinity
print((df2 == float('-inf')).sum())  # Count negative infinity

In [ ]:
# Drop rows with NaN values
df2 = df2.dropna()

In [ ]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests

# Select the variables to test for Granger causality with 'Condition' column
variables = ['TP2', 'TP3', 'H1', 'DV_pressure', 'Reservoirs', 'Oil_temperature', 'Motor_current', 'COMP', 'DV_eletric', 'Towers', 'MPG', 'LPS', 'Pressure_switch', 'Oil_level', 'Caudal_impulses']

# Perform the Granger causality test for each variable with 'Condition' as the dependent variable
results = {}
for var in variables:
    test_result = grangercausalitytests(df2[['condition', var]], maxlag=5, verbose=True)  # maxlag depends on your dataset
    results[var] = test_result

# This will output whether each variable Granger-causes 'Condition' based on p-values.


In [ ]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests

def check_exogenous_granger(df, target_column, exogenous_columns, max_lag=5, significance_level=0.05):
    for column in exogenous_columns:
        if column in df.columns:
            # Perform Granger Causality test
            test_result = grangercausalitytests(df[[target_column, column]], max_lag, verbose=False)
            
            # Extract the p-values from the test results
            p_values = [test_result[lag][0]['ssr_chi2test'][1] for lag in range(1, max_lag+1)]
            
            # If the p-value is greater than the significance level, the variable is exogenous
            if all(p > significance_level for p in p_values):
                print(f"{column} is considered exogenous with respect to {target_column}.")
            else:
                print(f"{column} Granger causes {target_column}.")
        else:
            print(f"{column} is not present in the dataset.")

target_column = 'condition'  
check_exogenous_granger(df2, target_column, variables)

In [ ]:
df2_clean = df2.drop(columns=['timestamp']).apply(pd.to_numeric, errors='coerce').dropna()

In [ ]:
from statsmodels.tsa.api import VAR

model = VAR(df2_clean)  # your aggregated DataFrame with causal vars + condition
lag_selection = model.select_order(20)  # test up to 20-minute lag
print(lag_selection.summary())

In [ ]:
model_fitted = model.fit(20)
print(model_fitted.summary())

In [ ]:
forecast = model_fitted.forecast(df2_clean.values[-20:], steps=5)  # next 5 minutes
print(pd.DataFrame(forecast, columns=df2_clean.columns))

In [ ]:
# Ensure forecast is a DataFrame (if not already)
forecast_df = pd.DataFrame(forecast, columns=df2_clean.columns)

In [ ]:
import matplotlib.pyplot as plt

# Ensure forecast is a DataFrame (if not already)
forecast_df = pd.DataFrame(forecast, columns=df2_clean.columns)

# Create x-values for the plot
x_actual = range(len(df2_clean) - 20, len(df2_clean))  # Last 20 actual data points
x_forecast = range(len(df2_clean), len(df2_clean) + 5)  # Forecasted next 5 points

# Plot actual values (last 20 points) for 'condition'
plt.plot(x_actual, df2_clean['condition'].iloc[-20:], label="Actual Values", color='blue')

# Plot forecasted values for the next 5 steps
plt.plot(x_forecast, forecast_df['condition'], label="Forecasted Values", color='red')

plt.xlabel('Time')
plt.ylabel('Condition')
plt.title('Forecast vs Actual Condition')
plt.legend()
plt.show()

In [ ]:
print(df2_clean['condition'].iloc[-20:])
print(forecast_df['condition'])